In [ ]:
# imports
! pip install pycaret
import pandas as pd
import numpy as np
from ast import literal_eval
import pycaret
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from pycaret.classification import *

In [ ]:
%load_ext autoreload
%autoreload
! pip install openai
! pip install tiktoken
# imports
import pandas as pd
import tiktoken
import openai
from openai.embeddings_utils import get_embedding

openai.api_key  ="sk-"

In [ ]:
# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191


In [ ]:
data = []
with open("/content/train_text.txt", "r") as file:
    for line in file:
        row = line.strip().split("\t")
        data.append(row)

df_train = pd.DataFrame(data)

df_train=df_train.rename(columns={0: "Text"})


In [ ]:
df_test = pd.DataFrame(pd.read_csv("/content/test_text.txt", sep="\t", header= None))
df_test=df_test.rename(columns={0: "Text"})
df_test.head()

In [ ]:
encoding = tiktoken.get_encoding(embedding_encoding)

# omit reviews that are too long to embed
df_train["n_tokens"] = df_train.Text.apply(lambda x: len(encoding.encode(x)))
top_n = 1000

df_train = df_train[df_train.n_tokens <= max_tokens].tail(top_n)
len(df_train)

In [ ]:
df_train["embedding"] = df_train.Text.apply(lambda x: get_embedding(x, engine=embedding_model))
df_train.to_csv("sentiment_embedding_train.csv")

In [ ]:
df_train

In [ ]:
df_train.columns

In [ ]:
df_train["embedding"] = df_train.embedding.apply(literal_eval).apply(np.array)  # convert string to array


In [ ]:
from ast import literal_eval
import numpy as np
df_train = df_train.dropna(subset=["embedding"]) 
df_train["embedding"] = df_train["embedding"].apply(lambda x: np.array(literal_eval(x)) if isinstance(x, str) else x)

print(df_train["embedding"].head())


In [ ]:
df_train

In [ ]:
df_exploded = df_train['embedding'].apply(pd.Series)

# Optionally, if the original DataFrame contains other columns, you can concatenate them with the exploded DataFrame
df_exploded = pd.concat([df_train.drop('embedding', axis=1), df_exploded], axis=1)


In [ ]:
df_exploded

In [ ]:
df_label= pd.DataFrame(pd.read_csv("/content/train_labels.txt", sep="\t", header= None))
df_train['label']=df_label[0]

In [ ]:
df_test=pd.read_csv("/content/sentiment_embedding.csv")
df_test=pd.DataFrame(df_test)

In [ ]:
df_test

In [ ]:
df_test.drop(columns=['Unnamed: 0','n_tokens'], inplace=True)

In [ ]:
from ast import literal_eval
import numpy as np

# Step 1: Handle missing values (if needed)
df_test = df_test.dropna(subset=["embedding"])  # Remove rows with NaN values in "embedding"

# Step 2: Convert strings to NumPy arrays
df_test["embedding"] = df_test["embedding"].apply(lambda x: np.array(literal_eval(x)) if isinstance(x, str) else x)

# Now, the "embedding" column should contain NumPy arrays
print(df_test["embedding"].head())
df_test_ex = df_test['embedding'].apply(pd.Series)

# Optionally, if the original DataFrame contains other columns, you can concatenate them with the exploded DataFrame
df_test = pd.concat([df_test.drop('embedding', axis=1), df_test_ex], axis=1)


In [ ]:
df_label= pd.DataFrame(pd.read_csv("/content/train_labels.txt", sep="\t", header= None))
df_exploded['label']=df_label[0]

In [ ]:
df_test

In [ ]:
df_exploded

In [ ]:
s= setup(df_exploded,test_data=df_test, target="label")

In [ ]:
best= compare_models()

In [ ]:
evaluate_model(best)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    list(df.embedding.values), df.label, test_size=0.2, random_state=42
)

# train random forest classifier
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
probas = clf.predict_proba(X_test)

report = classification_report(y_test, preds)
print(report)
